In [1]:
!pip install --quiet openai pandas


In [21]:
import os
import json
import pandas as pd
from datetime import datetime
from typing import List, Dict

import openai

# ==== НАСТРОЙКИ YANDEX CLOUD ====
# ВСТАВЬ СВОЙ API-KEY и FOLDER_ID

YANDEX_CLOUD_API_KEY = "наш api YANDEX CLOUD"
YANDEX_CLOUD_FOLDER_ID = "наш FOLDER_ID"
YANDEX_CLOUD_MODEL = "yandexgpt-lite"

client = openai.OpenAI(
    api_key=YANDEX_CLOUD_API_KEY,
    base_url="https://rest-assistant.api.cloud.yandex.net/v1",
    project=YANDEX_CLOUD_FOLDER_ID,
)

print("YandexGPT клиент сконфигурирован.")


YandexGPT клиент сконфигурирован.


In [19]:
LABELS = [
    "Запрос информации/документов",
    "Официальная жалоба или претензия",
    "Регуляторный запрос",
    "Партнёрское предложение",
    "Запрос на согласование",
    "Уведомление или информирование",
]

BASE_SYSTEM_PROMPT = """
Ты генерируешь официальные деловые письма, связанные с банковской сферой
(клиенты, корпоративные клиенты, партнёры, регуляторы, внутренние служебные письма банка).

Требования к письмам:
- язык: строгий литературный русский;
- стиль: официально-деловой, но живой, без канцелярита ради канцелярита;
- тематика: банковские продукты, договоры, реквизиты, регуляторные требования, мероприятия, партнёрство;
- длина каждого письма: примерно 800–2000 символов (1–5 абзацев).

ОЧЕНЬ ВАЖНО:
- НЕ начинай все письма одинаково ("Добрый день!", "Уважаемый банк" и т.п.).
  Вариируй обращения: "Уважаемые коллеги", "Уважаемый Председатель Правления",
  "В адрес вашего Банка обращаемся с...", "Настоящим направляем..." и т.д.
- НЕ используй одну и ту же фразу "Просим предоставить..." во всех письмах.
  Вариируй формулировки: "прошу направить", "запрашиваем", "обращаемся с просьбой",
  "настоящим просим", "будем признательны за", "ожидаем от вас".
- Меняй роли отправителей: юрлица, ИП, госорганы, фонд, дочерние общества, департаменты банка.
- Меняй типы ситуаций: разные продукты, разные риски, разные периоды и суммы.

Формат ответа: СТРОГО JSON-массив, где каждый элемент — объект:
{
  "text": "<полный текст письма без меток 'Запрос...' и без комментариев>",
  "label": "<один из 6 классов>"
}
Без пояснений, без комментариев, только валидный JSON.
"""


LABELS = [
    "Запрос информации/документов",
    "Официальная жалоба или претензия",
    "Регуляторный запрос",
    "Партнёрское предложение",
    "Запрос на согласование",
    "Уведомление или информирование",
]

BASE_SYSTEM_PROMPT = """
Ты генерируешь официальные деловые письма, связанные с банковской сферой
(клиенты, корпоративные клиенты, партнёры, регуляторы, внутренние служебные письма банка).

Требования к письмам:
- язык: строгий литературный русский;
- стиль: официально-деловой, но живой, без канцелярита ради канцелярита;
- тематика: банковские продукты, договоры, реквизиты, регуляторные требования, мероприятия, партнёрство;
- длина каждого письма: примерно 800–2000 символов (1–5 абзацев).

ОЧЕНЬ ВАЖНО:
- НЕ начинай все письма одинаково ("Добрый день!", "Уважаемый банк" и т.п.).
  Вариируй обращения: "Уважаемые коллеги", "Уважаемый Председатель Правления",
  "В адрес вашего Банка обращаемся с...", "Настоящим направляем..." и т.д.
- НЕ используй одну и ту же фразу "Просим предоставить..." во всех письмах.
  Вариируй формулировки: "прошу направить", "запрашиваем", "обращаемся с просьбой",
  "настоящим просим", "будем признательны за", "ожидаем от вас".
- Меняй роли отправителей: юрлица, ИП, госорганы, фонд, дочерние общества, департаменты банка.
- Меняй типы ситуаций: разные продукты, разные риски, разные периоды и суммы.

Формат ответа: СТРОГО JSON-массив, где каждый элемент — объект:
{
  "text": "<полный текст письма без меток 'Запрос...' и без комментариев>",
  "label": "<один из 6 классов>"
}
Без пояснений, без комментариев, только валидный JSON.
"""

CLASS_PROMPTS = {
    "Запрос информации/документов": """
Ты должен сгенерировать {n} РАЗНЫХ писем класса "Запрос информации/документов"
в адрес крупного банка.

Класс определяется так:
- отправитель просит банк предоставить информацию или документы;
- это МОЖЕТ быть резкий тон, если есть нарушение, но не обязательно;
- в письме всегда есть предмет запроса (что именно нужно) и, часто, срок.

Вот примеры СТИЛЯ (НЕ копируй их дословно, используй только как ориентир):

Пример 1 (строгий тон, одновременно запрос инфы и фактически претензия):
"Настоящим уведомляем о грубом нарушении условий договора №БС-1456 от 15.03.2023:
средства с нашего счёта были списаны без предварительного уведомления.
Требуем немедленного разъяснения и возврата средств в течение 3 рабочих дней."

Пример 2 (спокойный запрос документа):
"Просим предоставить выписку по расчётному счёту за III квартал 2025 года,
а также копию действующего тарифного плана по расчётно-кассовому обслуживанию."

Требования к сгенерированным письмам:
- ВСЕ письма относятся к классу "Запрос информации/документов".
- Каждый текст — уникальный по:
  * типу отправителя (ООО, АО, ИП, госорган, дочерняя компания, департамент);
  * продукту (расчётный счёт, кредит, депозит, карта, дистанционный сервис, отчётность);
  * структуре (иногда один абзац, иногда 3–4 абзаца, иногда маркированные пункты);
  * формулировкам (НЕ повторяй дословно обороты из примеров и своих предыдущих писем).
- В части писем может быть жёсткий тон, в части — нейтральный.
- Можно ссылаться на договоры, номера счетов, периоды, внутренние регламенты, указания ЦБ.

Сгенерируй массив JSON из {n} объектов.
Поле "label" у всех объектов должно быть ровно: "Запрос информации/документов".
""",

    "Официальная жалоба или претензия": """
Ты должен сгенерировать {n} РАЗНЫХ писем класса "Официальная жалоба или претензия"
в адрес крупного банка.

Класс определяется так:
- отправитель выражает недовольство действиями или бездействием банка;
- упоминаются нарушения условий договора, правил, регламентов или закона;
- формулируется ТРЕБОВАНИЕ: вернуть средства, устранить нарушение, дать ответ в срок.

Вот пример стиля (НЕ копируй дословно, только ориентир, не начинать всегда одинаково, использовать разные пример!!!):
"Настоящим уведомляем о грубом нарушении условий договора №БС-1456 от 15.03.2023:
средства с нашего счёта были списаны без предварительного уведомления. Требуем
немедленного разъяснения и возврата средств в течение 3 рабочих дней."

Требования:
- У каждого письма есть:
  * описание фактов (что произошло, когда, по какому договору/счёту);
  * обоснование (почему отправитель считает ситуацию нарушением — ссылка на условия договора,
    внутренние документы, закон или указание регулятора);
  * чётко сформулированное требование и, по возможности, срок удовлетворения требований.
- Тон может быть:
  * от сдержанно-официального до жёсткого, но без прямых угроз.
- Меняй:
  * тип клиента (МСБ, крупный корпоративный клиент, ИП, частное лицо),
  * продукт (кредиты, РКО, валютные операции, зарплатные проекты, интернет-банк),
  * тип проблемы (двойное списание, отказ в операции, ошибочное блокирование, неверное информирование).

Сгенерируй массив JSON из {n} объектов.
Поле "label" у всех объектов должно быть ровно: "Официальная жалоба или претензия".
""",

    "Регуляторный запрос": """
Ты должен сгенерировать {n} РАЗНЫХ писем класса "Регуляторный запрос"
от имени Банка России или иного надзорного органа (Росфинмониторинг, ФНС и т.п.)
в адрес коммерческого банка.

Класс определяется так:
- письмо исходит от регулятора/надзора;
- содержится требование предоставить информацию, отчётность, пояснения или документы;
- есть ссылка на нормы (указания, инструкции, методические рекомендации, статьи закона);
- указан срок предоставления информации.

Пример стиля (только ориентир, НЕ копировать дословно, не начинать всегда одинаково, использовать разные пример!!!):
"На основании п. 4.2 Указания Банка России №55-У от 10.04.2024 просим представить
информацию о сделках с признаками возможного отмывания денежных средств за III квартал 2025 года
до 25.11.2025 включительно."

Требования:
- В каждом письме:
  * ссылка на условное указание/инструкцию/нормативный акт (формат: №58-У, №123/2025, №03-11-25/00876 и т.п.);
  * описано, КАКИЕ именно данные нужны (по каким формам отчётности, периодам, продуктам, сегментам клиентов);
  * указан дедлайн в формате "не позднее", "до ... включительно".
- Меняй:
  * тип запроса (AML/отмывание, просроченная задолженность, качество кредитного портфеля,
    операции с нерезидентами, валютный контроль, отчётность по формам 0409… и т.д.);
  * глубину детализации (иногда короткий запрос, иногда развёрнутое письмо с перечнями).

Сгенерируй массив JSON из {n} объектов.
Поле "label" у всех объектов должно быть ровно: "Регуляторный запрос".
""",

    "Партнёрское предложение": """
Ты должен сгенерировать {n} РАЗНЫХ писем класса "Партнёрское предложение"
в адрес банка.

Класс определяется так:
- отправитель (компания, журнал, финтех, платёжный сервис, образовательный проект)
предлагает банку сотрудничество/партнёрство;
- описывается формат партнёрства и выгоды для банка;
- есть предложение обсудить условия, назначить встречу, дать обратную связь.

Примеры стиля (НЕ копировать дословно, использовать как ориентир, не начинать всегда одинаково, использовать разные пример!!!):
1) "Добрый день! Предлагаем установить партнёрство в рамках совместного запуска
цифровой платформы для МСБ. Прикрепляем презентацию и условия сотрудничества. Готовы обсудить детали."
2) Письмо про журнал "Финансовый Дом" с описанием тиража, аудитории, форматов статей и вебинара.

Требования:
- В каждом письме:
  * чётко указано, что именно предлагается (совместный продукт, медиапартнёрство, совместное мероприятие,
    исследование, образовательный проект и т.п.);
  * есть описание целевой аудитории/охвата (МСБ, розничные клиенты, премиальный сегмент и т.п.);
  * обозначены ожидаемые выгоды для банка (повышение узнаваемости, привлечение клиентов, укрепление имиджа и т.д.).
- Меняй:
  * тип отправителя (журнал, отраслевое СМИ, финтех-стартап, платёжный агрегатор, образовательная платформа);
  * формат партнёрства (спецвыпуск журнала, вебинар, конференция, кобрендинговый продукт);
  * тон — от более формального до дружелюбно-делового.

Сгенерируй массив JSON из {n} объектов.
Поле "label" у всех объектов должно быть ровно: "Партнёрское предложение".
""",

    "Запрос на согласование": """
Ты должен сгенерировать {n} РАЗНЫХ писем класса "Запрос на согласование"
(approval) в адрес банка или внутри группы компаний (дочерние/дочерние общества -> головной банк).

Класс определяется так:
- отправитель просит СОГЛАСОВАТЬ действия, документ, мероприятие, условия сделки или проект;
- перечисляются приложенные материалы/документы;
- указан желательный срок согласования.

Примеры стиля (НЕ копировать дословно, использовать как ориентир, не начинать всегда одинаково, использовать разные пример!!!):
1) Письмо о согласовании офлайн-мероприятия "Рога и копыта" для корпоративных клиентов
с программой, списком приглашённых и проектом коммуникационного письма.
2) Письмо о согласовании благотворительного проекта "Дети — наше будущее" с описанием охвата,
бюджета, приложенными документами фонда и сроками.

Требования:
- В каждом письме чётко указано:
  * что именно нужно согласовать (мероприятие, благотворительный проект, нестандартные условия по сделке,
    использование бренда банка, выпуск материала в СМИ и т.п.);
  * какие документы/материалы прилагаются (программа, макеты, драфт договора, бюджет, презентация);
  * к какому сроку желательно получить согласование.
- Меняй:
  * тип инициатора (дочерняя компания, департамент маркетинга, благотворительный фонд, проектный офис);
  * масштаб (локальное мероприятие, федеральная акция, пилотный проект);
  * контекст (корпоративные клиенты, МСБ, НКО, госструктуры).

Сгенерируй массив JSON из {n} объектов.
Поле "label" у всех объектов должно быть ровно: "Запрос на согласование".
""",

    "Уведомление или информирование": """
Ты должен сгенерировать {n} РАЗНЫХ писем класса "Уведомление или информирование"
от имени банка или регулятора.

Класс определяется так:
- письма носят информационный или уведомительный характер;
- чаще всего от получателя НЕ требуется ответа (или это явно оговорено);
- сообщаются факты: изменения реквизитов, методик, тарифов, запуск сервисов, плановые работы.

Примеры стиля (НЕ копировать дословно, использовать как ориентир, не начинать всегда одинаково, использовать разные пример!!!):
1) "Сообщаем об изменении реквизитов ООО 'ТехноЛайн' с 01.12.2025. Новые реквизиты прилагаются.
Просим актуализировать данные в вашей системе."
2) "Настоящим уведомляем вас об изменении методики расчёта показателя 'Доля просроченной задолженности...'
согласно Указанию Банка России №58-У..."

Требования:
- В каждом письме:
  * чётко указано, ЧТО именно изменилось или о чём информируют (реквизиты, методика расчёта, тарифы, режим работы сервисов);
  * при необходимости указана дата вступления изменений в силу;
  * может быть фраза "ответ на данное уведомление не требуется" или аналогичная.
- Меняй:
  * отправителя (банк -> клиенты/партнёры, регулятор -> банк, банк -> сотрудники/филиалы);
  * тип изменений (методика расчёта показателя, регламент работы, технические работы, запуск нового сервиса);
  * степень детализации (от короткого уведомления до подробного описания изменений).

Сгенерируй массив JSON из {n} объектов.
Поле "label" у всех объектов должно быть ровно: "Уведомление или информирование".
""",
}




In [16]:
def call_yandex_gpt(prompt: str) -> str:
    """
    Делает один запрос в YandexGPT Lite через Responses API
    и возвращает СЫРОЙ текстовый ответ (строка).
    """
    response = client.responses.create(
        model=f"gpt://{YANDEX_CLOUD_FOLDER_ID}/{YANDEX_CLOUD_MODEL}",
        input=[
            {"role": "system", "content": BASE_SYSTEM_PROMPT},
            {"role": "user", "content": prompt},
        ],
    )
    # В OpenAI-совместимом API у Яндекса итоговый текст лежит в output_text
    return response.output_text


def generate_emails_for_label(label: str, n: int = 10) -> pd.DataFrame:
    """
    Генерирует n писем для заданного класса label.
    Возвращает DataFrame с колонками text, label.
    """
    if label not in CLASS_PROMPTS:
        raise ValueError(f"Неизвестный label: {label}")

    user_prompt = CLASS_PROMPTS[label].format(n=n)
    raw = call_yandex_gpt(user_prompt)

    raw_stripped = raw.strip()

    # Убираем ``` если модель завернула JSON в кодовый блок
    if raw_stripped.startswith("```"):
        # отрезаем первый ``` и последний ```
        parts = raw_stripped.split("```")
        # между первым и последним как раз содержимое
        raw_stripped = "".join(parts[1:-1]).strip()

    # Пытаемся вытащить именно JSON-массив [ ... ]
    start = raw_stripped.find("[")
    end = raw_stripped.rfind("]")
    if start == -1 or end == -1:
        print("⚠ Не удалось найти JSON-массив в ответе. Сырой текст:")
        print(raw)
        raise ValueError("JSON array not found in model output")

    json_text = raw_stripped[start:end+1]

    # Парсим JSON
    try:
        data = json.loads(json_text)
    except json.JSONDecodeError as e:
        print("⚠ Не удалось распарсить JSON. Текст, который пытались распарсить:")
        print(json_text)
        raise e

    rows = []
    for item in data:
        text = item.get("text", "").strip()
        if not text:
            continue
        rows.append({"text": text, "label": label})

    df_batch = pd.DataFrame(rows)
    return df_batch


In [17]:
import os

DATA_PATH = "data/letters.csv"
os.makedirs("data", exist_ok=True)

def append_batch_to_csv(df_batch: pd.DataFrame):
    """
    Добавляет сгенерированные письма в data/letters.csv.
    Формат: text;label
    """
    # если файла ещё нет — создаём
    if os.path.exists(DATA_PATH):
        existing = pd.read_csv(DATA_PATH, sep=";")
        combined = pd.concat([existing, df_batch], ignore_index=True)
    else:
        combined = df_batch.copy()

    combined.to_csv(DATA_PATH, sep=";", index=False)
    print(f"✅ Записано {len(df_batch)} писем. Всего в датасете теперь: {len(combined)}")

    # Покажем парочку примеров
    print("\nПримеры последних записей:")
    display(combined.tail(5))


In [31]:
for label in LABELS:
    print(f"\n=== Генерируем {10} писем для класса: {label} ===")
    batch = generate_emails_for_label(label, n=10)
    append_batch_to_csv(batch)




=== Генерируем 10 писем для класса: Запрос информации/документов ===
✅ Записано 10 писем. Всего в датасете теперь: 730

Примеры последних записей:


,text,label
725,В адрес Вашего Банка обращаемся с запросом на предоставление информации о состоянии расчётов по счёту № 123456 за последние 3 месяца. Ожидаем подробный отчёт в течение 5 рабочих дней.,Запрос информации/документов
726,Просим предоставить информацию о доступных кредитных продуктах для малого бизнеса. Ожидаем подробное описание каждого предложения в течение 7 рабочих дней.,Запрос информации/документов
727,"В связи с проверкой со стороны налоговой службы, просим предоставить все документы, подтверждающие правильность расчётов по налогам. Ожидаем полный пакет документов в течение 10 рабочих дней.",Запрос информации/документов
728,Просим предоставить информацию о тарифах и комиссиях по обслуживанию банковских карт. Ожидаем подробное описание каждого тарифа в течение 5 рабочих дней.,Запрос информации/документов
729,В адрес Вашего Банка обращаемся с запросом на предоставление информации о доступных дистанционных сервисах. Ожидаем подробное описание каждого сервиса в течение 7 рабочих дней.,Запрос информации/документов



=== Генерируем 10 писем для класса: Официальная жалоба или претензия ===
✅ Записано 10 писем. Всего в датасете теперь: 740

Примеры последних записей:


,text,label
735,"Обращаемся с претензией по факту неверного информирования о состоянии нашего валютного счёта. Согласно условиям договора, банк обязан предоставлять точную и своевременную информацию о состоянии счёта. Однако предоставленная информация оказалась неверной, что привело к финансовым потерям. Требуем немедленного исправления ошибки и компенсации за причинённый ущерб.",Официальная жалоба или претензия
736,"Настоящим направляем официальную жалобу на нарушение условий договора №БС-1456 от 15.03.2023. Согласно пункту 6.1 договора, банк обязан осуществлять платежи в течение 1 рабочего дня. Однако платёж, осуществлённый 10 марта текущего года, был проведён с задержкой на 3 дня. Требуем немедленного устранения нарушения и компенсации за причинённый ущерб.",Официальная жалоба или претензия
737,"Обращаемся с претензией по факту отказа в проведении операции по интернет-банку. Согласно условиям договора, банк обязан осуществлять платежи по поручению клиента через интернет-банк. Отказ был мотивирован техническими причинами, однако подобные отказы происходят регулярно. Требуем немедленного устранения нарушений и компенсации за причинённые неудобства.",Официальная жалоба или претензия
738,"В адрес Вашего Банка обращаемся с официальной претензией на нарушение условий договора №БС-1456 от 15.03.2023. Согласно пункту 7.2 договора, банк обязан информировать клиента о любых изменениях в условиях обслуживания. Однако информация об изменении комиссии за обслуживание счёта была предоставлена с задержкой на 10 дней. Требуем немедленного информирования о всех изменениях и компенсации за причинённый ущерб.",Официальная жалоба или претензия
739,"Обращаемся с претензией по факту ошибочного блокирования нашего расчётного счёта №4100. Блокировка была осуществлена без объяснения причин и уведомления, что противоречит пункту 3.2 регламента обслуживания корпоративных клиентов. Требуем немедленного снятия блокировки и компенсации убытков, связанных с невозможностью проведения платежей.",Официальная жалоба или претензия



=== Генерируем 10 писем для класса: Регуляторный запрос ===
✅ Записано 10 писем. Всего в датасете теперь: 750

Примеры последних записей:


...

,text,label
755,Уважаемые коллеги! Предлагаем вашему банку сотрудничество в рамках организации вебинара для корпоративных клиентов. Вебинар будет посвящён актуальным вопросам управления финансами и инвестициями. Ожидаемые выгоды для банка — повышение узнаваемости бренда среди корпоративных клиентов и укрепление позиций на рынке финансовых услуг. Приглашаем вас обсудить условия партнёрства.,Партнёрское предложение
756,"В адрес вашего Банка обращаемся с предложением о партнёрстве в рамках разработки специального выпуска нашего журнала, посвящённого инновациям в банковской сфере. Специальный выпуск будет содержать аналитические материалы, интервью с экспертами и обзоры лучших практик. Ожидаемые выгоды для банка — повышение узнаваемости среди профессионалов финансового сектора и укрепление имиджа инновационного банка. Приглашаем вас обсудить детали сотрудничества.",Партнёрское предложение
757,"Уважаемые представители банка! Мы, платёжный сервис [Название], предлагаем вашему банку сотрудничество в рамках разработки и внедрения совместных программ лояльности для клиентов. Ожидаемые выгоды для банка — увеличение клиентской базы и повышение лояльности существующих клиентов. Приглашаем вас обсудить детали партнёрства и возможные выгоды.",Партнёрское предложение
758,"Добрый день! Предлагаем вашему банку стать информационным партнёром нашего отраслевого СМИ. Мы публикуем новости, аналитику и обзоры рынка, охватывая широкую аудиторию профессионалов финансового сектора. Ожидаемые выгоды для банка — повышение узнаваемости среди целевой аудитории и укрепление имиджа надёжного партнёра в финансовом секторе. Будем рады обсудить условия сотрудничества.",Партнёрское предложение
759,Уважаемые коллеги! Предлагаем вашему банку сотрудничество в рамках организации конференции для специалистов по кибербезопасности в банковской сфере. Конференция соберёт ведущих экспертов и предоставит уникальную возможность для обмена опытом и обсуждения актуальных угроз. Ожидаемые выгоды для банка — повышение узнаваемости бренда и укрепление деловых связей. Приглашаем вас обсудить детали партнёрства.,Партнёрское предложение



=== Генерируем 10 писем для класса: Запрос на согласование ===
✅ Записано 10 писем. Всего в датасете теперь: 770

Примеры последних записей:


,text,label
765,"Уважаемые коллеги,\n\nПросим согласовать проведение пилотного проекта по внедрению новой системы управления рисками. Проект включает в себя тестирование системы на [количество] объектах и анализ полученных результатов.\n\nК приложенным материалам относятся: проектная документация, план тестирования, расчёты.\n\nОжидаем ваше согласование до [дата].\n\nС уважением, [название проектного офиса]",Запрос на согласование
766,"Уважаемый банк,\n\nОбращаемся с просьбой согласовать условия сотрудничества с [название компании] для предоставления финансовых услуг корпоративным клиентам. В рамках сотрудничества планируется разработка и внедрение совместных продуктов.\n\nК приложенным материалам относятся: проект договора, описание продуктов, анализ рынка.\n\nПросим рассмотреть и согласовать условия до [дата].\n\nС уважением, [название департамента корпоративных клиентов]",Запрос на согласование
767,"Уважаемые коллеги,\n\nПросим согласовать проведение онлайн-мероприятия для сотрудников банка. Мероприятие запланировано на [дата], программа включает в себя [описание программы].\n\nК приложенным материалам относятся: программа мероприятия, список участников, проект коммуникационного письма.\n\nОжидаем ваше согласование до [дата].\n\nС уважением, [название департамента]",Запрос на согласование
768,"Уважаемый банк,\n\nОбращаемся с просьбой согласовать условия сделки по инвестированию в [название проекта]. В рамках сделки планируется вложение средств в размере [сумма] с целью поддержки [описание проекта].\n\nК приложенным материалам относятся: проект договора, описание проекта, расчёты.\n\nПросим рассмотреть и согласовать условия до [дата].\n\nС уважением, [название дочерней компании]",Запрос на согласование
769,"Уважаемые коллеги,\n\nПросим согласовать проведение благотворительной акции «Помощь пожилым» для поддержки пожилых людей. Акция включает в себя организацию сбора средств и оказание адресной помощи.\n\nК приложенным материалам относятся: описание акции, бюджет, план мероприятий.\n\nОжидаем ваше согласование до [дата].\n\nС уважением, [название благотворительного фонда]",Запрос на согласование



=== Генерируем 10 писем для класса: Уведомление или информирование ===
✅ Записано 10 писем. Всего в датасете теперь: 780

Примеры последних записей:


,text,label
775,"Уважаемые сотрудники, информируем вас о том, что с 01.01.2026 изменяются правила начисления бонусов по зарплатным картам. Новые условия начисления бонусов будут опубликованы на внутреннем портале. Просим вас ознакомиться с изменениями до начала нового года.",Уведомление или информирование
776,В адрес вашего Банка обращаемся с уведомлением об изменении порядка идентификации клиентов при онлайн-операциях. Новые требования вступают в силу с 01.02.2026. Просим вас ознакомиться с изменениями и провести необходимые корректировки в системах.,Уведомление или информирование
777,"Уважаемые клиенты, информируем вас о том, что с 01.03.2026 изменяются условия обслуживания по депозитам. Новые процентные ставки и сроки размещения будут опубликованы на официальном сайте банка. Просим вас ознакомиться с изменениями до указанной даты.",Уведомление или информирование
778,Сообщаем вам о запуске нового сервиса — онлайн-консультаций с банковскими специалистами. Консультации доступны через личный кабинет на сайте банка. Будем признательны за распространение информации среди ваших клиентов.,Уведомление или информирование
779,"Уважаемые коллеги, информируем вас об изменении порядка обработки платежей через систему быстрых платежей. Новые правила вступают в силу с 01.04.2026. Просим вас ознакомиться с изменениями и провести необходимые корректировки в системах.",Уведомление или информирование


In [32]:
import pandas as pd

DATA_PATH = "data/letters.csv"

# 1. Загрузка
df = pd.read_csv(DATA_PATH, sep=";")

print("=== Общая информация по датасету ===")
print(f"Всего строк: {len(df)}")
print(f"Колонки: {list(df.columns)}\n")

# 2. Проверим, что есть нужные колонки
assert "text" in df.columns, "В датасете нет колонки 'text'"
assert "label" in df.columns, "В датасете нет колонки 'label'"

# 3. Количество уникальных классов и распределение
print("=== Распределение по классам (label.value_counts) ===")
label_counts = df["label"].value_counts()
print(label_counts, "\n")

print("Всего уникальных классов:", df["label"].nunique())
print("Список уникальных классов:")
for lbl in sorted(df["label"].unique()):
    print(" -", lbl)
print()

# 4. Покажем по 2 примера для каждого класса (целиком)
pd.set_option("display.max_colwidth", None)

for lbl in sorted(df["label"].unique()):
    print("=" * 100)
    print(f"КЛАСС: {lbl}")
    print("=" * 100)

    subset = df[df["label"] == lbl].head(2)
    if subset.empty:
        print("❌ Нет примеров для этого класса\n")
        continue

    for idx, row in subset.iterrows():
        print(f"\n--- Пример index = {idx} ---")
        print(row["text"])
        print("\n" + "-" * 100)

    print("\n")


=== Общая информация по датасету ===
Всего строк: 780
Колонки: ['text', 'label']

=== Распределение по классам (label.value_counts) ===
label
Запрос информации/документов        130
Официальная жалоба или претензия    130
Регуляторный запрос                 130
Партнёрское предложение             130
Запрос на согласование              130
Уведомление или информирование      130
Name: count, dtype: int64 

Всего уникальных классов: 6
Список уникальных классов:
 - Запрос информации/документов
 - Запрос на согласование
 - Официальная жалоба или претензия
 - Партнёрское предложение
 - Регуляторный запрос
 - Уведомление или информирование

КЛАСС: Запрос информации/документов

--- Пример index = 0 ---
Обращаемся с просьбой предоставить информацию о текущем состоянии нашего кредитного договора № 12345 от 01.01.2023, включая текущий остаток задолженности и график платежей. Срок предоставления — до конца текущего рабочего дня.

------------------------------------------------------------------